# 待看参考文献

* 消除随机相位偏移：you are facing the Mona Lisa: spot localization using PHY layer information

# WIFI 无线通信知识

## 带通采样定理

带通采样的核心是 **频谱搬移**，通过采样带来的频率移位，将高频的带通信号移动到低频采样区

设带通信号的频率范围是 $[f_L, f_H]$，带宽：$B = f_H - f_L$

使用采样频率 $f_s$ 进行采样后，频谱会以 $f_s$ 发生偏移，假设向右移动 m 个 fs 后逼近 $f_L$，为了不发生混叠，要求满足：

$$
-f_L + m\times f_s \le f_L 
$$

同时，在 $f_H$ 处，也有要求：

$$
-f_H + (m+1)\times f_s \ge f_H
$$

联合起来我们有：

$$
\frac{2f_H}{m+1} \le fs \le \frac{2f_L}{m}
$$

此外，我们还需要有：

$$
\frac{2f_H}{m+1} \le \frac{2f_L}{m}
$$

求解得到：$m = \frac{f_L}{B}$，m为非负正整数，其含义可以理解为，在 $f_L$ 里面，可以整整放下多少个带宽，为求出最小采样频率 $\frac{2f_H}{m+1}$，我们看到：

$$
f_H = (m+1)B + k\times B
$$

$f_H$所能装下的带宽数只比 $f_L$ 多一个，我们可以记 $n = m + 1 $，上面的 k 表示小数个带宽，则最小的采样频率为：

$$
f_s = \frac{2f_H}{m+1} = 2B(1 + \frac{k}{n})
$$

# EArth Mover's Distance 度量不同直方图文件之间的距离

EMD 用来衡量两个概率分布之间的距离，其量化了把 `概率分布1` 转变成 `概率分布2` 所需要的最小 “成本”



假设

In [5]:
from pyemd import emd
import numpy as np

# Define two histograms (probability distributions)
hist1 = np.array([0.2, 0.3, 0.5])  # Example histogram 1
hist2 = np.array([0.1, 0.6, 0.3])  # Example histogram 2

# Define the ground distance matrix (cost matrix)
# This matrix represents the cost of transporting mass from one bin to another.
# It should have the same dimensions as the histograms.
cost_matrix = np.array([
    [1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0]
])

# Calculate the EMD between the two histograms using the specified cost matrix
emd_value = emd(hist1, hist2, cost_matrix)

# Print the EMD value
print("Earth Mover's Distance:", emd_value)


Earth Mover's Distance: 0.3


# 相位校准

实际获得的相位：

$$
\theta_k = \angle H(f_k) + 2\pi w_k (\Delta t) +2\pi (\Delta w) t +\delta_k
$$

$2\pi w_k (\Delta t) $ 和 $2\pi (\Delta w) t$ 分别代表时钟同步误差和载波频率偏移引起的随机相位偏移，$\delta_k$ 表示测量噪声。

**文献\[32\]** 采用线性变换进行相位校准：

$$
\theta_k^{'}= \theta_k -\alpha w_k -\beta
$$

其中， $\alpha$ 和 $\beta$ 分别为随所有子载波变化的斜率和偏移量

# 基于 CSI 的室内动态人员检测方案

**输入**：静态无人场景下采集的 CSI 方差概率分布直方图 $W_static$，信号接收端实时采集到的 CSI 原始数据

**输出**：室内是否存在动态目标

**假设**：一个发送天线 + 3根接收天线

**算法过程**

```python
for i = range(3):
    提取 CSI 的幅度信息 ;
    异常点去除 + 加权移动平均滤波；
    计算信号时域主成分 Pi，i 为载波
    for j in range(1,m): # 舍弃第一个主成分，只要后面的 m-1 个主成分
        提取第 j 个主成分的方差概率分布直方图 Wj；
        计算 Wj 与 静态特征直方图的距离 EMDj = EMD(Wstatic, Wj)
    
    计算第 i 根天线的环境异常度： ai = sum(EMDj), j in range(1,m)

if median(ai) > threshold: # threshold 是实现确定的阈值
    print("室内存在动态目标")
else:
    print("室内不存在动态目标")
```

# 基于离散小波变换的去噪方案

参考文献：**\[133\]**

小波去噪是一种建立在小波变换多分辨率分析基础上的新兴算法，主要分为三个阶段：`小波分解`、`基于阈值的小波系数量化处理`、`小波重构`

- **小波分解**

小波分解阶段，通过小波变换将信号递归分解到多个频带上，分解出的小波系数包括：**高频细节系数** 和**低频近似系数**；

- **量化处理**

对不同频带上的小波细节系数进行阈值量化处理

- **重构**

利用小波逆变换将最低一层的低频近似系数和处理后的高频细节系数重新组合得到去噪后的信号

# 基于 CSI 的室内静态人员检测方案 R-PSHD

**假设**： 检测区域内存在单组 AP-MP 链路，构成 1X3 MIMO 无线信道

**输入**： 信号接收端实时采集到的 CSI 原始数据

**输出**： 室内是否存在静态目标

**算法描述**：

```python
count = 0 # 统计不同接收天线组合的检测结果
提取 CSI 的相位信息 CSI_pha

for i in [1, 2]:
    for j in [2, 3]: # 遍历所有的接收天线的排列组合
        CSI_phase_diff = CSI_phase[i] - CSI_phase[j] // 天线相位差
        根据异常点密度设置检测窗口大小，进行异常点去除；
        利用局部去均值滤除直流成分；
        利用离散小波变换带通滤波，滤除无关成分
        根据 绝对中位差MAD(CSI_phase_diff) 进行子载波选取
        
        for k in range(0,m): # 假定选择了 m 个子载波
            计算峰值间隔 Tk = [t1, t2, ..., tp]
            计算呼吸速率 bpm_k = 60*f_sample / median(Tk)
        
        bpm_ave = mean(bpm_k), k in range(1, m) # m 个载波的呼吸速率均值
        if bpm_ave >= bpm_min and bpm_ave <= bpm_max: #符合正常的呼吸速率
            count = count + 1
if count > 3/2:
    print("室内存在静态目标")
else:
    print("室内无人")
        

```